In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as snb
import tensorflow as tf
from tensorflow import keras
from scipy import stats as st

In [6]:
# Returns wether PC os is linux or other
def isLinux():
    return os.name == 'posix'

In [7]:
biosense_readings_path = ''
emstast_readings_path = ''

if isLinux():
    pass
else:
    biosense_readings_path = 'C:\\Users\\pveso\\Downloads\\Biosense_Validation\\Commertial_Electrodes_Biosense\\'
    emstast_readings_path = 'C:\\Users\\pveso\\Downloads\\Biosense_Validation\\Commertial_Electrodes_Emstat_Blue\\Readings\\'

In [8]:
# reading all csv files
readings_list = sorted(os.listdir(biosense_readings_path))
df_list_rhodamine = []
df_list_virgin = []
if (len(readings_list) > 0):
    for i in range(len(readings_list)):
        file = readings_list[i]
        df = pd.read_csv(biosense_readings_path + file)
        if ('data_R' in file):
            df_list_rhodamine.append(df)
        else:
            df_list_virgin.append(df)

    print(f'reading pandas df successifully\nRhodamine:{len(df_list_rhodamine)}\nVirgin:{len(df_list_virgin)}')
else:
    print('There is no file in the path')   

reading pandas df successifully
Rhodamine:25
Virgin:25


In [9]:
# Joining all pandas files into groups with rhodamine and no-rhodamin
biosense_rhodamine = pd.concat(df_list_rhodamine, axis=1)
biosense_virgin = pd.concat(df_list_virgin, axis=1)
potential_rhodamine = biosense_rhodamine.iloc[:,0]
currents_rhodamine = biosense_rhodamine.iloc[:,1::2]
potential_virgin = biosense_virgin.iloc[:,0]
currents_virgin = biosense_virgin.iloc[:,1::2]

In [21]:
# Rhodamine = 1
# Virgin = 0

Y = np.concatenate((np.zeros(shape = (1, currents_virgin.shape[1])), np.ones(shape = (1, currents_rhodamine.shape[1]))), axis = 1)
Y = Y.transpose()
Y.shape

(50, 1)

In [22]:
X = np.concatenate((currents_virgin, currents_rhodamine), axis = 1)
X = X.transpose()
X.shape

(50, 1115)

In [29]:
# normalization min-max
for i in range(len(X)):
    values = X[i,:]
    min_ = np.min(values)
    max_ = np.max(values)
    X[i,:] = (values - min_) / (max_ - min_)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(33, 1115)
(33, 1)
(17, 1115)
(17, 1)


In [38]:
model = tf.keras.Sequential()

# First convolutional layer
model.add(tf.keras.layers.Conv1D(activation='relu', input_shape = (1115, 1), kernel_size=7, filters=16))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

# Second convolutional layer
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

# Flatten layer to transition from convolutional layers to dense layers
model.add(tf.keras.layers.Flatten())

# Fully connected layers
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()
          

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 1109, 16)          128       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 554, 16)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 552, 32)           1568      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 276, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8832)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               4522496   
_________________________________________________________________
dense_5 (Dense)              (None, 256)              

In [39]:
# Reshape the data to fit the 1D CNN input shape
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
batch_size = 5
epochs = 50

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

Train on 33 samples, validate on 17 samples
Epoch 1/50
33/33 [==============================] - 0s 13ms/sample - loss: 0.7377 - acc: 0.3939 - val_loss: 0.6880 - val_acc: 0.5294
Epoch 2/50
33/33 [==============================] - 0s 7ms/sample - loss: 0.7040 - acc: 0.4242 - val_loss: 0.6891 - val_acc: 0.4706
Epoch 3/50
33/33 [==============================] - 0s 7ms/sample - loss: 0.6879 - acc: 0.4242 - val_loss: 0.6760 - val_acc: 0.9412
Epoch 4/50
33/33 [==============================] - 0s 7ms/sample - loss: 0.6910 - acc: 0.5758 - val_loss: 0.6694 - val_acc: 0.4706
Epoch 5/50
33/33 [==============================] - 0s 7ms/sample - loss: 0.6401 - acc: 0.6667 - val_loss: 0.6246 - val_acc: 0.5294
Epoch 6/50
33/33 [==============================] - 0s 8ms/sample - loss: 0.6515 - acc: 0.6061 - val_loss: 0.5515 - val_acc: 1.0000
Epoch 7/50
33/33 [==============================] - 0s 7ms/sample - loss: 0.5541 - acc: 0.7576 - val_loss: 0.4542 - val_acc: 1.0000
Epoch 8/50
33/33 [=============

In [45]:
# Matriz confusão
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
y_pred = y_pred > 0.5
confusion_matrix(y_test, y_pred)

array([[8, 0],
       [0, 9]], dtype=int64)

In [ ]:
# Organizando 